# Deploying Campaigns and Filters (AWS Console)

In this notebook, we'll deploy our trained recommendation models and define some filtering rules, using the [Amazon Personalize console UI](https://console.aws.amazon.com/personalize/home).

> For an **alternative** approach to the same steps same steps *programmatically* using [Boto3, the AWS SDK for Python](https://boto3.amazonaws.com/v1/documentation/api/latest/index.html) - see Notebook [04b_Deploying_Campaigns_and_Filters_(Python_SDK).ipynb](04b_Deploying_Campaigns_and_Filters_(Python_SDK).ipynb) instead.

⚠️ You'll need to already have run the previous notebooks in this series to set up your environment and train **solutions** (models) in Amazon Personalize, including **waiting for your solution versions to finish training**

## Create campaigns

A campaign is a hosted solution version: an endpoint which you can query for recommendations in real-time. Pricing is determined by throughput capacity in **transactions per second** (TPS).

When deploying a campaign you select a minimum **provisioned** TPS, which the endpoint will retain resources to serve with low-latency. However, the campaign will **auto-scale** above this if required by traffic volumes. As with many applications, autoscaling events may see some additional latency while extra infrastructure is being spun up. For more information, see the [pricing page](https://aws.amazon.com/personalize/pricing/).

While provisioning ahead may be important for serving large-scale and latency-critical demand, for initial PoCs and demos like this it will make sense for us to select the minimum provisioned TPS (1).

Let's start deploying the campaigns.

### User Personalization

For our first campaign, you can get started from the dataset group dashboard page.

▶️ **Click** *Create new campaign* to start the workflow

![](static/imgs/dashboard-solutions-trained.png "Dataset group dashboard screen showing option to create campaign")

▶️ **Configure** your campaign as follows:

- **Name** it `personalize-movielens-up`
- Select the **user personalization solution** from the dropdown

The other settings (including the ID of your trained solution version) should populate by default. Note that for this recipe (which includes functionality to "cold-start" recommendations for items with no or limited interactions in the historical data) we have additional parameters to control the trade-off between "exploring" these cold items and sticking to our ML-driven recommendations.

![](static/imgs/create-campaign-up.png "Create campaign screen for user-personalization solution")

When you're ready, go ahead and click **Create campaign**.

> ⏰ It can take around 10 minutes to deploy a campaign. You can check the status of your campaigns from the "Campaigns" tab in the sidebar menu.

...But there's **no need for us to wait yet**: Let's move ahead with creating our other campaigns.

![](static/imgs/campaigns-list-up-only.png "Campaigns list view showing U-P campaign creating")

### SIMS

From the *Campaigns* tab of the sidebar as shown above, click the **Create campaign** button.

▶️ **Configure** your campaign as follows:

- **Name** it `personalize-movielens-sims`
- Select the **SIMS solution** from the dropdown

This time the screen is a little simpler, as the sims recipe has no deployment-time options:

![](static/imgs/create-campaign-sims.png "Create campaign screen for SIMS solution")

When you're ready, go ahead and click **Create campaign**.

### Personalized Ranking

As before, from the *Campaigns* tab of the sidebar, click the **Create campaign** button.

▶️ **Configure** your campaign as follows:

- **Name** it `personalize-movielens-rerank`
- Select the **re-ranking solution** from the dropdown

This time the screen is a little simpler, as the sims recipe has no deployment-time options:

![](static/imgs/create-campaign-sims.png "Create campaign screen for SIMS solution")

When you're ready, go ahead and click **Create campaign**.

## Wait for Campaigns to Deploy

▶️ **Wait** until all of your campaigns show as 'Active' on the Campaigns list view:

![](static/imgs/campaigns-list-all-active.png "Campaigns list view showing 3 active campaigns")

## Using Campaigns from the AWS Console *(Information Only)*

As you may notice, it's possible to **'Test campaign results'** from the console for active campaigns, including applying the filter expressions we'll create next!

![](static/imgs/test-campaign-up-input.png "Campaign detail screen showing 'test campaign results' form")

...However, we only see recommended item IDs and scores (without linking back to metadata such as title) - so we'll instead focus on using the Python SDK to explore recommendations in the next notebook, for something a bit more human-readable:

![](static/imgs/test-campaign-up-results.png "Campaign detail screen showing sample results - ID and score columns")

## Create Event Trackers

**Why** are we focussing on real-time model deployment with campaigns to start with, rather than batch inference? Well, recommendations can be most valuable when they're **fresh and dynamic** - updating to respond to a user's recent interactions and interests.

To get this dynamic behaviour, we'll have to **notify Personalize of new events (interactions)** as they happen - and to do that, we'll need to create an **Event Tracker**.

You'll see how this tracker gets used in the next notebook, but for now:

▶️ **Open** the *Event trackers* tab of the sidebar menu to go to the (empty) event trackers list

![](static/imgs/trackers-list-empty.png "Trackers list screen with no trackers created")

▶️ **Click** the *Create event tracker* button and **name** your event tracker, like `personalize-movielens-tracker`

![](static/imgs/create-tracker.png "Create tracker screen with name configured")

▶️ **Copy** the **tracking ID** from the next screen, and replace it below before running the cell to save it:

In [ ]:
tracking_id = "12345678-abcd-7890-cdef-1234567890ab"  # REPLACE ME WITH YOURS!
%store tracking_id

The rest of this screen gives guidance on integrating the created tracker with your applications - but since we'll just be making SDK calls direct from a Python notebook... You can go ahead and click **Finish**

![](static/imgs/tracker-created.png "Tracker creation 'install the SDK' screen")

We'll see the event tracker in action in the next notebook!

## Create Filters

**Filters** allow us to implement **business rules** to post-process the recommendations generated by our ML models, before they're returned from the campaign API.

As further detailed in the [documentation](https://docs.aws.amazon.com/personalize/latest/dg/filter.html), filters can serve a range of use-cases such as:

- Filtering based on item metadata (such as genre or year)
- Filtering based on past interactions (such as excluding items the user has already watched)
- Filtering based on user metadata and combinations (such as excluding certain genres based on the customer's age or membership tier)

...Using whatever metadata fields we added to our datasets.

Filters are written in a SQL-like [Filter expression language](https://docs.aws.amazon.com/personalize/latest/dg/filter-expressions.html) - which we'll see in action here.

Let's create some example filters using the fields in our sample dataset. Start by navigating to the **Filters** tab on the sidebar menu:

![](static/imgs/filters-list-empty.png "Filters list screen (empty)")

### 1970s Movies

For our first attempt, we'll create a filter for showing only movies from the 1970s.

▶️ **Click** *Create a new filter* to start the workflow

▶️ **Configure** your filter as follows:

- **Name** it `1970s`
- Enter the **filter expression** below - either directly or using the expression builder tool:

```
INCLUDE ItemID WHERE Items.YEAR >= 1970 AND Items.YEAR < 1980
```

![](static/imgs/create-filter-1970s.png "Create filter screen for 1970s movies expression")

When you're ready, go ahead and click **Create filter** to finish.


### 'Watch Again' and 'Try Something New'

Next let's create two opposite filters: One to show only movies our user has watched (reviewed) before, and one to show only movies they haven't.


▶️ **Click** *Create a new filter* to start the workflow, as we did before

▶️ **Configure** your filter as follows:

- **Name** it `watched`
- Enter the **filter expression** below - either directly or using the expression builder tool:

```
INCLUDE ItemID WHERE Interactions.event_type IN ("review")
```

> ⚠️ **Remember** that our sample dataset only has "review" events, but a typical video-on-demand use-case would have multiple kinds of events recorded!

![](static/imgs/create-filter-watched.png "Create filter screen showing 'watched' filter expression")

When you're ready, go ahead and click **Create filter** to finish.

▶️ **Repeat** the process to create a filter called `unwatched` with **EXCLUDE** instead of **INCLUDE** in the filter expression

### Choose a Genre

Finally, we'll look at a basic example including a **parameter** you can set at request time: Building a filter for recommending from a single genre.

▶️ **Click** *Create a new filter* to start the workflow, as we did before

▶️ **Configure** your filter as follows:

- **Name** it `by-genre`
- Enter the **filter expression** below - either directly or using the expression builder tool:

```
INCLUDE ItemID WHERE Items.GENRES IN ($GENRE)
```

Once you're finished, you should have 4 filters as below:

![](static/imgs/filters-list-full.png "Filters list screen showing created filters")

## All set!

We've now deployed our campaigns and created some filter expressions to help further refine our results.

In the next notebook we'll dip back in to Python to demonstrate **using** these models from application code to generate recommendations!

Follow along in [05_Interacting_with_Campaigns_and_Filters.ipynb](05_Interacting_with_Campaigns_and_Filters.ipynb)